# Script del proyecto Poliza de Seguros


# Scrip1 : Codigo de Entrenamiento

In [2]:
import pandas as pd
import statsmodels.api as sm
import pickle

In [3]:
# Cargar la tabla
poliza_df = pd.read_csv("data/raw/poliza_train.csv")
#- Se definen las variables 
X_train = poliza_df[['edad', 'AniosDireccion', 'Gastocoche', 'Aniosempleo', 'Aniosresiden']]
X_train = sm.add_constant(X_train)
y_train = poliza_df[['ingres']]

In [4]:
# Entrenamos el modelo con toda la muestra
M_R = sm.OLS(y_train,X_train)
M_R = M_R.fit()

In [5]:
# Guardamos el modelo entrenado para usarlo en produccion
filename = 'models/best_model.pkl'
pickle.dump(M_R, open(filename, 'wb'))

# Scrip2 : Codigo de Validacion

In [6]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import pickle
import matplotlib.pyplot as plt
from sklearn.metrics import *

from sklearn.metrics import mean_squared_error
from sklearn import metrics

In [7]:
# Cargar la tabla
poliza_test_df = pd.read_csv("data/raw/poliza_val.csv")
#- Se definen las variables 
X_test = poliza_test_df[['edad', 'AniosDireccion', 'Gastocoche', 'Aniosempleo', 'Aniosresiden']]
X_test = sm.add_constant(X_test)
y_test = poliza_test_df[['ingres']]

In [8]:
# Leemos el modelo entrenado!
filename = 'models/best_model.pkl'
model = pickle.load(open(filename, 'rb'))

In [9]:
# Predecimos sobre el set de datos de implementacion con el modelo entrenado
y_pred_test=model.predict(X_test)

In [10]:
## Metricas de validación
def calc_metrics(y_test,y_pred_test):
    print('MAPE: ',      metrics.mean_absolute_error(y_test, y_pred_test))  
    print('MAE: ',       metrics.mean_squared_error(y_test, y_pred_test))  
    print('RMSE: ',  np.sqrt(metrics.mean_squared_error(y_test, y_pred_test)))
    rmse_ols = np.sqrt(metrics.mean_squared_error(y_test, y_pred_test))
    print(f"El error (RMSE) de test es: {rmse_ols}")

In [11]:
calc_metrics(y_test,y_pred_test)

MAPE:  19.46082156960554
MAE:  2229.9720042890576
RMSE:  47.22257939046805
El error (RMSE) de test es: 47.22257939046805


# Scrip3 : Codigo de Scoring (Automatización)

In [12]:
import pandas as pd
import statsmodels.api as sm
import pickle

In [13]:
# Cargar la tabla
poliza_scor_df = pd.read_csv("data/raw/poliza_scor.csv")

In [14]:
# Leemos el modelo entrenado!
filename = 'models/best_model.pkl'
model = pickle.load(open(filename, 'rb'))

In [15]:
# Predecimos sobre el set de datos de implementacion con el modelo entrenado
scores=model.predict(poliza_scor_df)

In [16]:
# Exportamos el resultado del modelo para cargarlo en el Feature Store o Data Mart de Modelos
# Le asignamos nombres a las columnas
df_score = pd.DataFrame(scores, columns=['PREDICT'])
# Exportamos la solucion
df_score.to_csv('data/scores/final_score.csv')